In [1]:


!pip install SVM



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for SVM: filename=svm-0.1.0-py3-none-any.whl size=3508 sha256=e79b02806d2085590dc1dfda1786083e6c3c467cc186f0caef785dd7792f6a63
  Stored in directory: c:\users\dhuor\appdata\local\pip\cache\wheels\f7\71\d5\55cef69f03651f4585848ac42c589f36e5d326c433257c9da7
Successfully built SVM



***Movie Genre Classification using TF-IDF and Naive Bayes***

Aim is to build a machine learning model to predict the genre of a movie based on its plot summary or textual information. To achieve this, we will leverage natural language processing (NLP) techniques, specifically TF-IDF (Term Frequency-Inverse Document Frequency), in conjunction with the Naive Bayes classification algorithm.

The primary goal is to create a model that can automatically assign one or more genres to a movie based on the textual description provided. This can be valuable for categorizing and organizing movies in databases, recommendation systems, and content filtering.

Importing Libaries


In [7]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report



**Load Dataset**

In [11]:
df = pd.read_csv('Movie Genre Dataset/train_data.txt', sep='\t', header=None)
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Movie Genre Dataset/train_data.txt'